# Video Notes AI Summary Troubleshooting

This notebook helps diagnose and troubleshoot issues with AI summary generation in the video notes processing workflow.

## Problem Description
You ran the following command but are not seeing AI summaries generated:
```bash
dotnet run -- video-notes -i "C:\Users\danshue.REDMOND\OneDrive\Education\MBA-Resources\Value Chain Management\Operations Management\operations-management-organization-and-analysis\01_course-orientation-operations-strategy\Live Session" --force --config "D:\source\notebook-automation-csharp-migration\config.json" --debug
```

The markdown files are being created with metadata, but the AI summary section is missing.

## Diagnostic Steps
We'll systematically check each component that could affect AI summary generation.

## 1. Setup Environment

First, let's verify the .NET environment and project structure.

In [ ]:
# Check .NET version
dotnet --version

# Verify we're in the correct directory
Get-Location

# Check if the NotebookAutomation solution builds successfully
Set-Location "D:\source\notebook-automation-csharp-migration"
dotnet build src/c-sharp/NotebookAutomation.sln

## 2. Load and Analyze Configuration File

Let's examine the configuration file to check AI-related settings.

In [ ]:
# Load and display the configuration file
$configPath = "D:\source\notebook-automation-csharp-migration\config.json"
if (Test-Path $configPath) {
    Write-Host "Configuration file exists at: $configPath"
    $config = Get-Content $configPath | ConvertFrom-Json
    $config | ConvertTo-Json -Depth 10
    
    # Check for AI-related settings
    Write-Host "`n=== AI Configuration Check ==="
    if ($config.AI) {
        Write-Host "AI section found in config"
        $config.AI | ConvertTo-Json -Depth 5
    } else {
        Write-Host "❌ No AI section found in configuration"
    }
    
    # Check for OpenAI API key
    if ($config.AI.OpenAIApiKey) {
        $keyLength = $config.AI.OpenAIApiKey.Length
        Write-Host "✅ OpenAI API Key found (length: $keyLength characters)"
    } elseif ($env:OPENAI_API_KEY) {
        $keyLength = $env:OPENAI_API_KEY.Length
        Write-Host "✅ OpenAI API Key found in environment variable (length: $keyLength characters)"
    } else {
        Write-Host "❌ No OpenAI API Key found in config or environment"
    }
} else {
    Write-Host "❌ Configuration file not found at: $configPath"
}

## 3. Parse Input Arguments and Analyze Directory

Let's examine the input directory and verify video files and transcripts.

In [ ]:
# Analyze the input directory
$inputPath = "C:\Users\danshue.REDMOND\OneDrive\Education\MBA-Resources\Value Chain Management\Operations Management\operations-management-organization-and-analysis\01_course-orientation-operations-strategy\Live Session"

if (Test-Path $inputPath) {
    Write-Host "✅ Input directory exists: $inputPath"
    Write-Host "`n=== Directory Contents ==="
    Get-ChildItem $inputPath -Recurse | Format-Table Name, Extension, Length, LastWriteTime
    
    # Check for video files
    $videoExtensions = @(".mp4", ".mov", ".avi", ".mkv", ".webm", ".wmv")
    $videoFiles = Get-ChildItem $inputPath -Recurse | Where-Object { $videoExtensions -contains $_.Extension.ToLower() }
    
    Write-Host "`n=== Video Files Found ==="
    if ($videoFiles.Count -gt 0) {
        $videoFiles | ForEach-Object {
            Write-Host "📹 $($_.Name) ($([math]::Round($_.Length/1MB, 2)) MB)"
            
            # Check for associated transcript files
            $baseName = [System.IO.Path]::GetFileNameWithoutExtension($_.Name)
            $transcriptTxt = Join-Path $_.DirectoryName "$baseName.txt"
            $transcriptMd = Join-Path $_.DirectoryName "$baseName.md"
            
            if (Test-Path $transcriptTxt) {
                Write-Host "   📄 Transcript found: $([System.IO.Path]::GetFileName($transcriptTxt))"
            } elseif (Test-Path $transcriptMd) {
                Write-Host "   📄 Transcript found: $([System.IO.Path]::GetFileName($transcriptMd))"
            } else {
                Write-Host "   ❌ No transcript found for this video"
            }
        }
    } else {
        Write-Host "❌ No video files found in the directory"
    }
    
    # Check for any existing markdown notes
    $markdownFiles = Get-ChildItem $inputPath -Recurse -Filter "*.md"
    Write-Host "`n=== Existing Markdown Files ==="
    if ($markdownFiles.Count -gt 0) {
        $markdownFiles | ForEach-Object {
            Write-Host "📝 $($_.Name)"
        }
    } else {
        Write-Host "No existing markdown files found"
    }
} else {
    Write-Host "❌ Input directory not found: $inputPath"
}

## 4. Test AI Summarizer Functionality

Let's create a simple test to verify the AI Summarizer is working correctly.

In [ ]:
# Test the AI Summarizer with a simple test
Set-Location "D:\source\notebook-automation-csharp-migration\src\c-sharp\NotebookAutomation.Cli"

Write-Host "Testing AI Summarizer functionality..."
Write-Host "This will attempt to generate a summary using the configured AI service."
Write-Host ""

# Create a temporary test text file
$testText = @"
This is a test transcript for an operations management course. 
The session covers key concepts including supply chain optimization, 
inventory management strategies, and lean manufacturing principles. 
Students learn about process improvement methodologies and how to 
analyze operational efficiency in modern business environments.
"@

$tempFile = [System.IO.Path]::GetTempFileName()
$testText | Out-File -FilePath $tempFile -Encoding UTF8

try {
    # Run a simple summarization test using the CLI
    dotnet run -- test-summarizer --input $tempFile --config "D:\source\notebook-automation-csharp-migration\config.json" --debug
} catch {
    Write-Host "❌ Error running AI summarizer test: $($_.Exception.Message)"
} finally {
    # Clean up temp file
    if (Test-Path $tempFile) {
        Remove-Item $tempFile
    }
}

## 5. Debug Video Notes Command Execution

Let's run the actual video notes command with verbose logging to see what's happening.

In [ ]:
# Run the video notes command with maximum verbosity
Set-Location "D:\source\notebook-automation-csharp-migration\src\c-sharp\NotebookAutomation.Cli"

Write-Host "Running video notes command with debug logging..."
Write-Host "Command: dotnet run -- video-notes -i `"C:\Users\danshue.REDMOND\OneDrive\Education\MBA-Resources\Value Chain Management\Operations Management\operations-management-organization-and-analysis\01_course-orientation-operations-strategy\Live Session`" --force --config `"D:\source\notebook-automation-csharp-migration\config.json`" --debug"
Write-Host ""

try {
    # Set environment variable for more detailed logging
    $env:ASPNETCORE_ENVIRONMENT = "Development"
    $env:Logging__LogLevel__Default = "Debug"
    
    # Run the command and capture output
    dotnet run -- video-notes -i "C:\Users\danshue.REDMOND\OneDrive\Education\MBA-Resources\Value Chain Management\Operations Management\operations-management-organization-and-analysis\01_course-orientation-operations-strategy\Live Session" --force --config "D:\source\notebook-automation-csharp-migration\config.json" --debug 2>&1
} catch {
    Write-Host "❌ Error running video notes command: $($_.Exception.Message)"
}

Write-Host "`n=== Log Analysis ==="
# Check for any log files that might have been created
$logPath = "D:\source\notebook-automation-csharp-migration\logs"
if (Test-Path $logPath) {
    Write-Host "Checking log files..."
    Get-ChildItem $logPath -Filter "*.log" | Sort-Object LastWriteTime -Descending | Select-Object -First 3 | ForEach-Object {
        Write-Host "📄 Latest log: $($_.Name) ($(Get-Date $_.LastWriteTime))"
        Write-Host "Last 20 lines:"
        Get-Content $_.FullName | Select-Object -Last 20
        Write-Host "---"
    }
} else {
    Write-Host "No log directory found at: $logPath"
}

## 6. Examine Generated Output

Let's look at the markdown files that were created and analyze their content.